## Exercise 1

Total cost that take to fetch the data using command:
```sql
EXPLAIN SELECT * FROM public.customer
```

Output:
```
Seq Scan on customer  (cost=0.00..4034.00 rows=100000 width=211)
```

For 3 different query:

1)
```sql
EXPLAIN ANALYZE SELECT * 
FROM public.customer
WHERE id>10 AND id<100;
```
```
Seq Scan on customer  (cost=0.00..328.00 rows=90 width=211)
```
2)
```sql
EXPLAIN SELECT * 
FROM public.customer
WHERE name = 'Jason Lopez';
```
```
Seq Scan on customer  (cost=0.00..4534.00 rows=2 width=211)
```
3)
```sql
EXPLAIN SELECT * 
FROM public.customer
WHERE address = 'USS Myers';
```
```
Seq Scan on customer  (cost=0.00..4784.00 rows=1 width=211)
```

Creating name index by following command:
```sql
CREATE INDEX idx_customer_name ON customer(name) 
```

Afer Indexing:
1)

```
Index Scan using customer_pkey on customer  (cost=0.29..12.16 rows=87 width=211) (actual time=0.009..0.041 rows=89 loops=1)
```

2)

```
->  Bitmap Index Scan on idx_customer_name  (cost=0.00..4.43 rows=2 width=0)
```

3)
```
Index Scan using idx_customer_address on customer  (cost=0.42..8.44 rows=1 width=211)
```


## Exercise 2

### 1-TASK

```sql
SELECT * FROM category as cat, film_category as f_c, film as f, inventory as i
WHERE (cat.name = 'Sci-Fi' or cat.name = 'Horror')
	AND cat.category_id = f_c.category_id
	AND f_c.film_id = f.film_id
    AND (f.rating = 'R' OR f.rating = 'PG-13')
	AND NOT EXISTS (SELECT * FROM rental WHERE i.inventory_id = rental.inventory_id)
```

ANALYZING:

```
"Nested Loop  (cost=606.27..693.69 rows=47 width=492)"
"  ->  Hash Join  (cost=606.00..627.00 rows=125 width=108)"
"        Hash Cond: (f_c.category_id = cat.category_id)"
"        ->  Seq Scan on film_category f_c  (cost=0.00..16.00 rows=1000 width=12)"
"        ->  Hash  (cost=605.97..605.97 rows=2 width=96)"
"              ->  Nested Loop  (cost=510.99..605.97 rows=2 width=96)"
"                    ->  Hash Anti Join  (cost=510.99..604.71 rows=1 width=16)"
"                          Hash Cond: (i.inventory_id = rental.inventory_id)"
"                          ->  Seq Scan on inventory i  (cost=0.00..70.81 rows=4581 width=16)"
"                          ->  Hash  (cost=310.44..310.44 rows=16044 width=4)"
"                                ->  Seq Scan on rental  (cost=0.00..310.44 rows=16044 width=4)"
"                    ->  Seq Scan on category cat  (cost=0.00..1.24 rows=2 width=80)"
"                          Filter: (((name)::text = 'Sci-Fi'::text) OR ((name)::text = 'Horror'::text))"
"  ->  Index Scan using film_pkey on film f  (cost=0.28..0.53 rows=1 width=384)"
"        Index Cond: (film_id = f_c.film_id)"
"        Filter: ((rating = 'R'::mpaa_rating) OR (rating = 'PG-13'::mpaa_rating))"
```

### 2-Task

```sql
WITH store_sum AS (SELECT st.*, SUM(pay.amount) FROM payment as pay
				  JOIN staff AS s ON pay.staff_id = s.staff_id
				  JOIN store AS st ON s.store_id = st.store_id
				  WHERE TO_DATE('020107', 'MMDDYY') < pay.payment_date 
				   AND pay.payment_date < TO_DATE('030107', 'MMDDYY')
				  GROUP BY st.store_id)
SELECT * FROM store_sum AS s
JOIN (SELECT ad.city_id, MAX(s.sum) FROM store_sum AS s
	 INNER JOIN address AS ad ON s.address_id = ad.address_id
	 GROUP BY ad.city_id) 
	 AS tot ON s.sum = tot.max
```

ANALYZING:

```
"Hash Join  (cost=458.74..458.82 rows=2 width=82)"
"  Hash Cond: ((max(s_1.sum)) = s.sum)"
"  CTE store_sum"
"    ->  HashAggregate  (cost=442.26..442.28 rows=2 width=48)"
"          Group Key: st.store_id"
"          ->  Hash Join  (cost=2.12..431.53 rows=2145 width=22)"
"                Hash Cond: (pay.staff_id = s_2.staff_id)"
"                ->  Seq Scan on payment pay  (cost=0.00..399.92 rows=2145 width=8)"
"                      Filter: ((to_date('020107'::text, 'MMDDYY'::text) < payment_date) AND (payment_date < to_date('030107'::text, 'MMDDYY'::text)))"
"                ->  Hash  (cost=2.09..2.09 rows=2 width=20)"
"                      ->  Nested Loop  (cost=0.00..2.09 rows=2 width=20)"
"                            Join Filter: (s_2.store_id = st.store_id)"
"                            ->  Seq Scan on staff s_2  (cost=0.00..1.02 rows=2 width=6)"
"                            ->  Materialize  (cost=0.00..1.03 rows=2 width=16)"
"                                  ->  Seq Scan on store st  (cost=0.00..1.02 rows=2 width=16)"
"  ->  GroupAggregate  (cost=16.39..16.42 rows=2 width=34)"
"        Group Key: ad.city_id"
"        ->  Sort  (cost=16.39..16.39 rows=2 width=34)"
"              Sort Key: ad.city_id"
"              ->  Hash Join  (cost=0.07..16.38 rows=2 width=34)"
"                    Hash Cond: (ad.address_id = s_1.address_id)"
"                    ->  Seq Scan on address ad  (cost=0.00..14.03 rows=603 width=6)"
"                    ->  Hash  (cost=0.04..0.04 rows=2 width=34)"
"                          ->  CTE Scan on store_sum s_1  (cost=0.00..0.04 rows=2 width=34)"
"  ->  Hash  (cost=0.04..0.04 rows=2 width=48)"
```

